In [11]:
import finpy_tse as fin
His = fin.Get_Price_History(stock = 'خودرو', start_date = '1400-01-01', end_date='1401-02-01', ignore_date = True,
                            adjust_price = True, show_weekday = True, double_date = True)
His.head()

,Date,Weekday,Open,High,Low,Close,Final,Volume,Value,No,Ticker,Name,Market,Adj Open,Adj High,Adj Low,Adj Close,Adj Final
J-Date,,,,,,,,,,,,,,,,,,
1380-01-05,2001-03-25,Sunday,2798,2802,2798,2800,2800,110870,310368080,115,خودرو,ایران خودرو,بورس,54,54,54,54,54
1380-01-06,2001-03-26,Monday,2801,2810,2797,2798,2798,96613,270595190,98,خودرو,ایران خودرو,بورس,54,54,54,54,54
1380-01-07,2001-03-27,Tuesday,2795,2800,2795,2795,2795,166600,466405131,120,خودرو,ایران خودرو,بورس,54,54,54,54,54
1380-01-08,2001-03-28,Wednesday,2800,2849,2796,2849,2849,80676,227566666,104,خودرو,ایران خودرو,بورس,54,55,54,55,55
1380-01-11,2001-03-31,Saturday,2840,2848,2800,2838,2838,177362,500068371,161,خودرو,ایران خودرو,بورس,55,55,54,55,55


In [12]:
import pandas as pd
His.columns

Index(['Date', 'Weekday', 'Open', 'High', 'Low', 'Close', 'Final', 'Volume',
       'Value', 'No', 'Ticker', 'Name', 'Market', 'Adj Open', 'Adj High',
       'Adj Low', 'Adj Close', 'Adj Final'],
      dtype='object')

In [13]:
His = His[['Date','Close', 'Volume']]
His

,Date,Close,Volume
J-Date,,,
1380-01-05,2001-03-25,2800,110870
1380-01-06,2001-03-26,2798,96613
1380-01-07,2001-03-27,2795,166600
1380-01-08,2001-03-28,2849,80676
1380-01-11,2001-03-31,2838,177362
...,...,...,...
1401-12-06,2023-02-25,3108,1365523241
1401-12-07,2023-02-26,2993,1204339387
1401-12-08,2023-02-27,3232,2529399129


### Find Financial ratios For Iranian Stock

In [29]:
import finpy_tse as fin
import pandas as pd

# Define the tickers and weights for your portfolio
tickers = ['فملی']

# Retrieve financial data for each ticker in your portfolio
dfs = []
df = fin.Get_Price_History(stock='فملی', start_date='1400-01-01', end_date='1401-02-01', ignore_date=True,
                           adjust_price=True, show_weekday=True, double_date=True)
df = df[['Date', 'Close', 'Volume']]
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df = df.set_index('Date')
dfs.append(df)

# Combine the data for all tickers into a single dataframe
combined_df = pd.concat(dfs, axis=1)

# Calculate financial ratios for the portfolio
price = combined_df['Close'].fillna(method='ffill')
volume = combined_df['Volume'].fillna(0)
roa = (price.diff(1) / price.shift(1)).rolling(4).mean()
cfo = ((price - price.shift(1)) / volume).rolling(4).mean()
accrual = (cfo - roa).rolling(4).mean()
delta_roa = roa.diff(1)
delta_leverage = (price / price.shift(1)).rolling(4).apply(lambda x: x[-1] / x[0] - 1)
delta_liquidity = volume.diff(1).rolling(4).mean() / volume.rolling(4).mean()
delta_margin = (price.diff(1) / price.shift(1)).rolling(4).mean()
delta_turnover = ((price - price.shift(1)).abs() / ((price + price.shift(1)) / 2)).rolling(4).mean()
delta_outstanding = (volume / volume.shift(1)).rolling(4).apply(lambda x: x[-1] / x[0] - 1)

# Calculate the weighted average of each ratio
weighted_ratios = pd.DataFrame(index=combined_df.index)

ratio_df = pd.DataFrame({
    'ROA': roa.iloc[:],
    'CFO': cfo.iloc[:],
    'Accrual': accrual.iloc[:],
    'Delta_ROA': delta_roa.iloc[:],
    'Delta_Leverage': delta_leverage.iloc[:],
    'Delta_Liquidity': delta_liquidity.iloc[:],
    'Delta_Margin': delta_margin.iloc[:],
    'Delta_Turnover': delta_turnover.iloc[:],
    'Delta_Outstanding': delta_outstanding.iloc[:]
})
ratio_df = ratio_df.fillna(method='ffill')
ratio_df = ratio_df.fillna(method='bfill')

# show the resulting dataframe
ratio_df

,ROA,CFO,Accrual,Delta_ROA,Delta_Leverage,Delta_Liquidity,Delta_Margin,Delta_Turnover,Delta_Outstanding
Date,,,,,,,,,
2007-02-04,0.019915,3.506082e-06,-0.016348,-0.004745,-0.000129,-1.208799,0.019915,0.019719,19.880486
2007-02-05,0.019915,3.506082e-06,-0.016348,-0.004745,-0.000129,-1.208799,0.019915,0.019719,19.880486
2007-02-06,0.019915,3.506082e-06,-0.016348,-0.004745,-0.000129,-1.208799,0.019915,0.019719,19.880486
2007-02-07,0.019915,3.506082e-06,-0.016348,-0.004745,-0.000129,-1.208799,0.019915,0.019719,19.880486
2007-02-10,0.019915,3.506082e-06,-0.016348,-0.004745,-0.000129,-1.208799,0.019915,0.019719,19.880486
...,...,...,...,...,...,...,...,...,...
2023-02-25,0.051105,2.646030e-06,-0.038309,0.007790,-0.006955,0.021741,0.051105,0.049636,-0.435386
2023-02-26,0.051885,2.102545e-06,-0.046200,0.000780,-0.002461,0.211466,0.051885,0.050368,-0.039046
2023-02-27,0.036363,1.739737e-06,-0.045665,-0.015522,-0.009709,-0.109628,0.036363,0.035414,0.133002


# API: Alpha Vantage
    provided ratios by this API:
        
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TECHNOLOGY',
 'Industry': 'ELECTRONIC COMPUTERS',
 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US',
 'FiscalYearEnd': 'September',
 'LatestQuarter': '2022-12-31',
 'MarketCapitalization': '2321231118000',
 'EBITDA': '125287997000',
 'PERatio': '24.95',
 'PEGRatio': '2.75',
 'BookValue': '3.581',
 'DividendPerShare': '0.91',
 'DividendYield': '0.0063',
 'EPS': '5.88',
 'RevenuePerShareTTM': '24.08',
 'ProfitMargin': '0.246',
 'OperatingMarginTTM': '0.294',
 'ReturnOnAssetsTTM': '0.196',
 'ReturnOnEquityTTM': '1.479',
 'RevenueTTM': '387537011000',
 'GrossProfitTTM': '170782000000',
 'DilutedEPSTTM': '5.88',
 'QuarterlyEarningsGrowthYOY': '-0.105',
 'QuarterlyRevenueGrowthYOY': '-0.055',
 'AnalystTargetPrice': '168.21',
 'TrailingPE': '24.95',
 'ForwardPE': '23.09',
 'PriceToSalesRatioTTM': '5.51',
 'PriceToBookRatio': '44.63',
 'EVToRevenue': '5.92',
 'EVToEBITDA': '17.53',
 'Beta': '1.278',
 '52WeekHigh': '178.53',
 '52WeekLow': '123.98',
 '50DayMovingAverage': '140.56',
 '200DayMovingAverage': '147.2',
 'SharesOutstanding': '15821900000',
 'DividendDate': '2023-02-16',
 'ExDividendDate': '2023-02-10'
        


In [ ]:
import requests

# Define API endpoint and parameters
symbol = 'AAPL'
api_key = 'VBQK7O4DSYUTBN6S'
endpoint = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}'



# Make API request
response = requests.get(endpoint)

# Parse JSON data
data = response.json()

# Extract relevant financial ratios
pe_ratio = float(data['PERatio'])
ps_ratio = float(data['PriceToSalesRatioTTM'])
pb_ratio = float(data['PriceToBookRatio'])
ev_to_sales_ratio = float(data['EVToRevenue'])
#debt_to_equity = float(data['debtToEquity'])
#debt_to_assets = float(data['debtToAssets'])
net_debt_to_ebitda = float(data['EVToEBITDA'])

# Calculate financial ratios
roe = float(data["ReturnOnEquityTTM"])
#net_profit_margin = float(data["ProfitMargin"])
#asset_turnover = 1 / (1 / roe + debt_to_assets)
#financial_leverage = (1 + net_profit_margin - roe) / (roe - net_profit_margin) * (1 - debt_to_equity)

# Calculate DuPont analysis
#net_income = float(data['netIncome'])
gross_profit = float(data['GrossProfitTTM'])
revenue = float(data['RevenuePerShareTTM'])
#total_assets = float(data['totalAssets'])
#total_liabilities = float(data['totalDebt'])
#operating_income = float(data['operatingIncome'])
operating_margin = float(data["OperatingMarginTTM"])
roa = float(data["ReturnOnAssetsTTM"])

#net_profit_margin_dupont = net_income / revenue
#asset_turnover_dupont = revenue / total_assets
#financial_leverage_dupont = total_assets / total_liabilities
#roe_dupont = net_profit_margin_dupont * asset_turnover_dupont * financial_leverage_dupont
eps = float(data["EPS"])
dividend_yield = float(data["DividendYield"])

print(f"Financial ratios for {symbol}:")
print(f"PE ratio: {pe_ratio}")
print(f"PS ratio: {ps_ratio}")
print(f"PB ratio: {pb_ratio}")
print(f"EV/Sales ratio: {ev_to_sales_ratio}")
#print(f"Debt to equity ratio: {debt_to_equity}")
#print(f"Debt to assets ratio: {debt_to_assets}")
print(f"Net debt to EBITDA ratio: {net_debt_to_ebitda}")
print(f"ROE: {roe}")
#print(f"Net profit margin: {net_profit_margin}")
#print(f"Asset turnover: {asset_turnover}")
#print(f"Financial leverage: {financial_leverage}")

print(f"\nDuPont analysis for {symbol}:")
#print(f"Net profit margin: {net_profit_margin_dupont}")
#print(f"Asset turnover: {asset_turnover_dupont}")
#print(f"Financial leverage: {financial_leverage_dupont}")
#print(f"ROE: {roe_dupont}")


print(f"Operating margin: {operating_margin}")
print(f"ROA: {roa}")
print(f"Earnings per share: {eps}")
print(f"Dividend yield: {dividend_yield}")


# API: intrinio
https://docs.intrinio.com/documentation/python

# API: Yahoo Finance 


In [ ]:
import requests

url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/qu/quote/AAPL/financial-data"

headers = {
	"X-RapidAPI-Key": "605431b43amsha1c7bfce7b0e88fp16b321jsn92d90f3a16cf",
	"X-RapidAPI-Host": "yahoo-finance15.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

from https://rapidapi.com/sparior/api/yahoo-finance15/ for AAPL we have below as Output: 

In [ ]:
import json

data = '''
{
  "financialData": {
    "maxAge": 86400,
    "currentPrice": {
      "raw": 148.4528,
      "fmt": "148.45"
    },
    "targetHighPrice": {
      "raw": 195,
      "fmt": "195.00"
    },
    "targetLowPrice": {
      "raw": 118,
      "fmt": "118.00"
    },
    "targetMeanPrice": {
      "raw": 168.21,
      "fmt": "168.21"
    },
    "targetMedianPrice": {
      "raw": 172.5,
      "fmt": "172.50"
    },
    "recommendationMean": {
      "raw": 2,
      "fmt": "2.00"
    },
    "recommendationKey": "buy",
    "numberOfAnalystOpinions": {
      "raw": 38,
      "fmt": "38",
      "longFmt": "38"
    },
    "totalCash": {
      "raw": 51355000832,
      "fmt": "51.36B",
      "longFmt": "51,355,000,832"
    },
    "totalCashPerShare": {
      "raw": 3.246,
      "fmt": "3.25"
    },
    "ebitda": {
      "raw": 125287997440,
      "fmt": "125.29B",
      "longFmt": "125,287,997,440"
    },
    "totalDebt": {
      "raw": 111109996544,
      "fmt": "111.11B",
      "longFmt": "111,109,996,544"
    },
    "quickRatio": {
      "raw": 0.769,
      "fmt": "0.77"
    },
    "currentRatio": {
      "raw": 0.938,
      "fmt": "0.94"
    },
    "totalRevenue": {
      "raw": 387537010688,
      "fmt": "387.54B",
      "longFmt": "387,537,010,688"
    },
    "debtToEquity": {
      "raw": 195.868,
      "fmt": "195.87"
    },
    "revenuePerShare": {
      "raw": 24.084,
      "fmt": "24.08"
    },
    "returnOnAssets": {
      "raw": 0.19569999,
      "fmt": "19.57%"
    },
    "returnOnEquity": {
      "raw": 1.47943,
      "fmt": "147.94%"
    },
    "grossProfits": {
      "raw": 170782000000,
      "fmt": "170.78B",
      "longFmt": "170,782,000,000"
    },
    "freeCashflow": {
      "raw": 84729126912,
      "fmt": "84.73B",
      "longFmt": "84,729,126,912"
    },
    "operatingCashflow": {
      "raw": 109189996544,
      "fmt": "109.19B",
      "longFmt": "109,189,996,544"
    },
    "earningsGrowth": {
      "raw": -0.105,
      "fmt": "-10.50%"
    },
    "


In [30]:
import requests

# Define API endpoint and parameters
symbol = 'AAPL'
api_key = '0093a06c2199659b77c42c1409f2ed53'
endpoint = f'https://financialmodelingprep.com/api/v3/ratios/{symbol}?apikey={api_key}'

# Make API request
response = requests.get(endpoint)

# Parse JSON data
data = response.json()

#get earliest time
data=data[0]
data

{'symbol': 'AAPL',
 'date': '2022-09-24',
 'period': 'FY',
 'currentRatio': 0.8793560286267226,
 'quickRatio': 0.7094075930952969,
 'cashRatio': 0.15356340351469652,
 'daysOfSalesOutstanding': 56.400204905560855,
 'daysOfInventoryOutstanding': 8.07569806661716,
 'operatingCycle': 64.47590297217802,
 'daysOfPayablesOutstanding': 104.6852773031054,
 'cashConversionCycle': -40.209374330927375,
 'grossProfitMargin': 0.43309630561360085,
 'operatingProfitMargin': 0.30288744395528594,
 'pretaxProfitMargin': 0.30204043334482966,
 'netProfitMargin': 0.2530964070519973,
 'effectiveTaxRate': 0.16204461684424407,
 'returnOnAssets': 0.2829244092925685,
 'returnOnEquity': 1.9695887275023682,
 'returnOnCapitalEmployed': 0.6008713457059057,
 'netIncomePerEBT': 0.8379553831557559,
 'ebtPerEbit': 0.997203546639651,
 'ebitPerRevenue': 0.30288744395528594,
 'debtRatio': 0.8563535598361469,
 'debtEquityRatio': 5.961536943479634,
 'longTermDebtToCapitalization': 0.6613535965140913,
 'totalDebtToCapitalizat

In [31]:
# Liquidity ratios
current_ratio = data['currentRatio']
quick_ratio = data['quickRatio']
cash_ratio = data['cashRatio']

# Activity ratios
receivables_turnover = 365 / data['daysOfSalesOutstanding']
inventory_turnover = 365 / data['daysOfInventoryOutstanding']
payables_turnover = 365 / data['daysOfPayablesOutstanding']

# Profitability ratios
gross_profit_margin = data['grossProfitMargin']
operating_profit_margin = data['operatingProfitMargin']
net_profit_margin = data['netProfitMargin']
return_on_assets = data['returnOnAssets']
return_on_equity = data['returnOnEquity']

# Debt ratios
debt_ratio = data['debtRatio']
debt_equity_ratio = data['debtEquityRatio']
long_term_debt_to_capitalization = data['longTermDebtToCapitalization']
total_debt_to_capitalization = data['totalDebtToCapitalization']
interest_coverage = data['interestCoverage']

# Valuation ratios
price_earnings_ratio = data['priceEarningsRatio']
price_to_book_ratio = data['priceToBookRatio']
price_to_sales_ratio = data['priceToSalesRatio']
price_to_free_cash_flows_ratio = data['priceToFreeCashFlowsRatio']

# Display the ratios
print(f'Current ratio: {current_ratio:.2f}')
print(f'Quick ratio: {quick_ratio:.2f}')
print(f'Cash ratio: {cash_ratio:.2f}')
print(f'Receivables turnover: {receivables_turnover:.2f}')
print(f'Inventory turnover: {inventory_turnover:.2f}')
print(f'Payables turnover: {payables_turnover:.2f}')
print(f'Gross profit margin: {gross_profit_margin:.2f}')
print(f'Operating profit margin: {operating_profit_margin:.2f}')
print(f'Net profit margin: {net_profit_margin:.2f}')
print(f'Return on assets: {return_on_assets:.2f}')
print(f'Return on equity: {return_on_equity:.2f}')
print(f'Debt ratio: {debt_ratio:.2f}')
print(f'Debt equity ratio: {debt_equity_ratio:.2f}')
print(f'Long-term debt to capitalization: {long_term_debt_to_capitalization:.2f}')
print(f'Total debt to capitalization: {total_debt_to_capitalization:.2f}')
print(f'Interest coverage: {interest_coverage:.2f}')
print(f'Price/earnings ratio: {price_earnings_ratio:.2f}')
print(f'Price-to-book ratio: {price_to_book_ratio:.2f}')
print(f'Price-to-sales ratio: {price_to_sales_ratio:.2f}')
print(f'Price-to-free cash flows ratio: {price_to_free_cash_flows_ratio:.2f}')




Current ratio: 0.88
Quick ratio: 0.71
Cash ratio: 0.15
Receivables turnover: 6.47
Inventory turnover: 45.20
Payables turnover: 3.49
Gross profit margin: 0.43
Operating profit margin: 0.30
Net profit margin: 0.25
Return on assets: 0.28
Return on equity: 1.97
Debt ratio: 0.86
Debt equity ratio: 5.96
Long-term debt to capitalization: 0.66
Total debt to capitalization: 0.70
Interest coverage: 40.75
Price/earnings ratio: 24.44
Price-to-book ratio: 48.14
Price-to-sales ratio: 6.19
Price-to-free cash flows ratio: 21.89


In [32]:
import yfinance as yf

# Define the symbol
symbol = "AAPL"

# Define the start and end dates
start_date = "2021-01-01"
end_date = "2021-01-06"

# Fetch the data from Yahoo Finance
data = yf.download(symbol, start=start_date, end=end_date)

# Print the financial performance data
print(data)


[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-01-04  133.520004  133.610001  126.760002  129.410004  127.680000   
2021-01-05  128.889999  131.740005  128.429993  131.009995  129.258591   

               Volume  
Date                   
2021-01-04  143301900  
2021-01-05   97664900  


# IEX cloud
tokens from https://iexcloud.io/console/tokens

# 12 data
https://twelvedata.com/account/api-keys
statistic was free if u want to see other plan plz https://twelvedata.com/docs 

In [ ]:
import requests

symbol = 'AAPL'
api_key = '06991cf1672848bf94e852068c0cf476'

url = f'https://api.twelvedata.com/statistics?symbol={symbol}&apikey={api_key}'


response = requests.get(url)

data = response.json()


In [ ]:
data